In [1]:
import numpy as np
import tensorflow as tf

In [2]:
print(f"Tensorflow Version: {tf.__version__}")
print(f"Eager mode: {tf.executing_eagerly()}")

Tensorflow Version: 2.1.0
Eager mode: True


## Eager mode
### Basic tensor calculation

In [3]:
x = [[2,]]
m = tf.matmul(x, x)
print(m)

tf.Tensor([[4]], shape=(1, 1), dtype=int32)


In [4]:
a = tf.constant([[1, 2],
                [3, 4]])
b = tf.add(a, 1)
print(a)

tf.Tensor(
[[1 2]
 [3 4]], shape=(2, 2), dtype=int32)


In [5]:
c = tf.multiply(a, b)
print(c)

tf.Tensor(
[[ 2  6]
 [12 20]], shape=(2, 2), dtype=int32)


### Variable and Automatic differentiate calculation
- NOTE: The resource that hold by GradientTape will be released right after gradient method is called

In [6]:
v = tf.Variable(0.0)
print((v+1).numpy())

1.0


In [7]:
v.assign(5)
print(v)
v.assign_add(1)
print(v)

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=5.0>
<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=6.0>


In [8]:
# differentiate for variable
w = tf.Variable([[1.0]])
with tf.GradientTape() as t:
    loss = w*w

In [9]:
grad = t.gradient(loss, w)
print(grad)

tf.Tensor([[2.]], shape=(1, 1), dtype=float32)


In [10]:
# differentiate for constant
c = tf.Variable([[3.0]])
with tf.GradientTape() as t:
    t.watch(c)
    loss = c*c

In [11]:
grad = t.gradient(loss, c)
print(grad)

tf.Tensor([[6.]], shape=(1, 1), dtype=float32)


### Automatic differentiate calculation and Automatic trainning

In [12]:
(train_image, train_label), (test_image, test_label) = tf.keras.datasets.mnist.load_data()

In [13]:
train_image = tf.cast(tf.expand_dims(train_image, -1)/255, tf.float32)
train_label = tf.cast(train_label, tf.int64)

In [14]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_image, train_label))
train_dataset = train_dataset.shuffle(10000).batch(32)
train_dataset

<BatchDataset shapes: ((None, 28, 28, 1), (None,)), types: (tf.float32, tf.int64)>

In [15]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, [3,3], activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.Conv2D(32, [3,3], activation='relu'),
    tf.keras.layers.GlobalMaxPool2D(),
    tf.keras.layers.Dense(10),
])

In [16]:
# NOTE, automatic trainning, no need to compile...
optimizer = tf.keras.optimizers.Adam()
loss_func = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [18]:
# Get one batch from dataset
ob_image, ob_label = next(iter(train_dataset))
print(ob_image.shape)
print(ob_label.shape)

(32, 28, 28, 1)
(32,)


In [21]:
ob_pred = model(ob_image)
print(ob_pred.shape)

(32, 10)


In [22]:
def loss(model, x, y):
    y_pred = model(x)
    return loss_func(y, y_pred)

In [26]:
def train_step(model, images, labels):
    with tf.GradientTape() as t:
        loss_step = loss(model, images, labels)
    grads = t.gradient(loss_step, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

In [27]:
# NOTE, this is CPU calculation! Not GPU!
def train():
    for epoch in range(10):
        for (batch, (images, labels)) in enumerate(train_dataset):
            train_step(model, images, labels)
        print(f"Epoch {epoch} completed!")

In [28]:
train()

Epoch 0 completed!
Epoch 1 completed!
Epoch 2 completed!
Epoch 3 completed!
Epoch 4 completed!
Epoch 5 completed!
Epoch 6 completed!
Epoch 7 completed!
Epoch 8 completed!
Epoch 9 completed!


### Summary of tf.keras.metrics

In [29]:
m = tf.keras.metrics.Mean('accuracy')
print(m(10))
print(m(20))

tf.Tensor(10.0, shape=(), dtype=float32)
tf.Tensor(15.0, shape=(), dtype=float32)


In [30]:
m.result().numpy()

15.0

In [31]:
m.reset_states()

In [33]:
train_loss = tf.keras.metrics.Mean('train_loss')
train_accuracy= tf.keras.metrics.SparseCategoricalAccuracy('train_accuracy')